## Cookie Factory Optimization Problem
Alex Szpakiewicz, Sara Thibierge, Léonard Roussard

In [1]:
# Library imports
import pandas as pd

### Data Loading

In [2]:
def load_defects(file_path='defects.csv'):
    """
    Load defect information.
    Parameters:
    - file_path: str, the path to defect.csv .
    Returns:
    - defects: DataFrame, a pandas DataFrame containing defect information.
    """
    defects = pd.read_csv(file_path)
    return defects

In [3]:
defects_data = load_defects("defects.csv")
defects_data.head()

,x,class
0,355.449335,c
1,92.496236,a
2,141.876795,c
3,431.833902,c
4,435.028461,c


In [ ]:
# Define the roll dought length
roll_dought = 500

### Define Biscuit Class

In [4]:
class Biscuit:
    def __init__(self, size, value, defect_thresholds):
        """
        Initialize a Biscuit instance.

        Parameters:
        - size: int, the size of the biscuit.
        - value: int, the value of the biscuit.
        - defect_thresholds: dict, a dictionary specifying the maximum allowed defects for each class.
        """
        self.size = size
        self.value = value
        self.defect_thresholds = defect_thresholds

    def __str__(self):
        """
        Return a string representation of the Biscuit instance.
        """
        return f"Biscuit(size={self.size}, value={self.value}, thresholds={self.defect_thresholds})"


The biscuit manufacturing factory aims to produce 4 types of biscuits, which are :

- Biscuit 0 with a length of 4, a value 6, and maximum allowed defects as {′a′ : 4, ′b′ :
2, ′c′ : 3}
- Biscuit 1 with a length of 8, a value 12, and maximum allowed defects as {′a′ : 5, ′b′ :
4, ′c′ : 4}
- Biscuit 2 with a length of 2, a value 1, and maximum allowed defects as {′a′ : 1, ′b′ :
2, ′c′ : 1}
- Biscuit 3 with a length of 5, a value 8, and maximum allowed defects as {′a′ : 2, ′b′ :
3, ′c′ : 2}

In [6]:
biscuit_0 = Biscuit(size=4, value=6, defect_thresholds={'a': 4, 'b': 2, 'c': 3})
biscuit_1 = Biscuit(size=8, value=12, defect_thresholds={'a': 5, 'b': 4, 'c': 4})
biscuit_2 = Biscuit(size=2, value=1, defect_thresholds={'a': 1, 'b': 2, 'c': 1})
biscuit_3 = Biscuit(size=5, value=8, defect_thresholds={'a': 2, 'b': 3, 'c': 2})